In [1]:
import gensim
import os
import collections
import smart_open
import random
import multiprocessing
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from pprint import pprint
import numpy as np

In [2]:
conceptPairDict={}
errors=[]
conceptPairList=[]

def read_pair(fname):
    with smart_open.smart_open(fname) as f:
        for i, line in enumerate(f):
            #get the id for each concept paragraph
            splitted = line.decode("iso-8859-1").split("\t")
            if len(splitted)==3:
                conceptPairList.append([splitted[1], splitted[2].replace("\r\n", ""), 1])
#                 conceptPairDict[splitted[1]] = splitted[2].replace("\r\n", "")
            else:
                errors.append(splitted)

pair_file = "/home/hao/AnacondaProjects/MLOntology/ontHierarchy.txt"
read_pair(pair_file)

first2pairs = conceptPairList[10:15]
print(first2pairs)
print(len(conceptPairList))

[['37225000', '52860004', 1], ['159386001', '159385002', 1], ['233836002', '233835003', 1], ['233836002', '304914007', 1], ['224923003', '224717003', 1]]
502206


In [3]:
conceptNotPairDict={}
conceptNotPairList=[]

def read_not_pair(fname):
    with smart_open.smart_open(fname) as f:
        for i, line in enumerate(f):
            #get the id for each concept paragraph
            splitted = line.decode("iso-8859-1").split("\t")
            if len(splitted)==2:
                conceptNotPairList.append([splitted[0], splitted[1].replace("\r\n", ""), 0])
#                 conceptNotPairDict[splitted[1]] = splitted[2].replace("\r\n", "")
            else:
                errors.append(splitted)

notPair_file = "/home/hao/AnacondaProjects/MLOntology/taxNotPairs.txt"
read_not_pair(notPair_file)

# first2pairs = {k: conceptNotPairDict[k] for k in list(conceptNotPairDict)[10:15]}
first2pairs =conceptNotPairList[10:15]
print(first2pairs)
print(len(conceptNotPairList))

[['273187009', '272765000', 0], ['272877001', '272765000', 0], ['273216002', '272765000', 0], ['273125004', '272765000', 0], ['272973003', '272765000', 0]]
6166563


In [4]:
path = "/home/hao/AnacondaProjects/MLOntology/model0"

model = gensim.models.Doc2Vec.load(path)

inferred_vector = model.infer_vector(['congenital', 'prolong', 'rupture', 'premature', 'membrane', 'lung'])
pprint(model.docvecs.most_similar([inferred_vector], topn=10))

[('237267007', 0.6276803612709045),
 ('722912007', 0.6034080982208252),
 ('446466006', 0.5920654535293579),
 ('722913002', 0.5904892683029175),
 ('67798003', 0.5712212324142456),
 ('253745002', 0.5680616497993469),
 ('10759661000119108', 0.5673926472663879),
 ('277485007', 0.5651165246963501),
 ('177130000', 0.5642103552818298),
 ('312974005', 0.5603421926498413)]


In [ ]:
path = "D:/MLOntology/model1"

model = gensim.models.Doc2Vec.load(path)

inferred_vector = model.infer_vector(['congenital', 'prolong', 'rupture', 'premature', 'membrane', 'lung'])
pprint(model.docvecs.most_similar([inferred_vector], topn=10))

In [5]:
from sklearn import svm

feature_number = 1024

train_list_before=[]
train_label_list=[]
test_list_before=[]
test_label_list =[]

offset = 6000*80

for i in range(60):
    index = np.arange(100)     #generate numbers from 0 to 100
    np.random.shuffle(index)        #shuffle the 100 values
    index = [int(100*i+j + offset) for j in index]
    print(index[10:20])
    train_list_before.extend([conceptPairList[b] for b in index[0:40]]) 
    train_label_list.extend([1]*40)
    train_list_before.extend([conceptNotPairList[b] for b in index[40:80]])
    train_label_list.extend([0]*40)
    test_list_before.extend([conceptPairList[b] for b in index[80:90]])
    test_label_list.extend([1]*10)
    test_list_before.extend([conceptNotPairList[b] for b in index[90:100]])
    test_label_list.extend([0]*10)


    
train_list =[]
test_list = []

for line in train_list_before:
    if line[0] in model.docvecs and line[1] in model.docvecs:
        a= model.docvecs[line[0]]
        b= model.docvecs[line[1]]
        c = np.array((a, b))
#         train_list.append(np.reshape(c, feature_number)) 
        train_list.append(np.reshape(c, feature_number, order='F'))


test_list_ids={}
for i, line in enumerate(test_list_before):
    if line[0] in model.docvecs and line[1] in model.docvecs:
        a= model.docvecs[line[0]]
        b= model.docvecs[line[1]]
        c = np.array((a, b))
        test_list_ids[i] = (line[0], line[1])
#         test_list.append(np.reshape(c, feature_number))
        test_list.append(np.reshape(c, feature_number, order='F'))

   
  

[480039, 480021, 480087, 480096, 480071, 480062, 480054, 480061, 480097, 480063]
[480180, 480176, 480153, 480127, 480132, 480173, 480126, 480189, 480193, 480182]
[480279, 480222, 480221, 480277, 480290, 480264, 480255, 480280, 480212, 480253]
[480336, 480390, 480393, 480396, 480394, 480354, 480361, 480340, 480327, 480334]
[480492, 480464, 480482, 480424, 480439, 480445, 480452, 480499, 480474, 480496]
[480544, 480512, 480552, 480587, 480563, 480531, 480520, 480596, 480506, 480536]
[480654, 480684, 480689, 480611, 480673, 480679, 480697, 480646, 480612, 480682]
[480785, 480732, 480798, 480797, 480701, 480751, 480707, 480750, 480767, 480716]
[480871, 480899, 480816, 480823, 480802, 480831, 480875, 480838, 480801, 480841]
[480953, 480933, 480913, 480985, 480904, 480961, 480978, 480902, 480962, 480903]
[481022, 481013, 481018, 481039, 481058, 481031, 481059, 481005, 481032, 481025]
[481156, 481107, 481150, 481160, 481120, 481188, 481109, 481115, 481137, 481163]
[481208, 481259, 481234, 481

In [6]:
print(train_list[30:50])
print(train_label_list[30:50])
print(len(train_list))
print(len(test_list))

[array([ 0.08479115,  0.0663422 ,  0.05275482, ...,  0.12619175,
       -0.12809703, -0.17363998], dtype=float32), array([ 0.18267164,  0.11323926,  0.17254877, ...,  0.38802034,
       -0.02322777, -0.18750308], dtype=float32), array([ 0.10050754,  0.11395817,  0.05497552, ...,  0.11559462,
        0.07492956,  0.04896749], dtype=float32), array([ 0.09933073,  0.2667613 , -0.11126914, ...,  0.12529561,
       -0.0748793 , -0.09309307], dtype=float32), array([-0.15109827, -0.29729855,  0.01252204, ...,  0.04958273,
       -0.06732967, -0.07634298], dtype=float32), array([-0.02135331, -0.04509294,  0.05080427, ..., -0.03882974,
       -0.06732488, -0.13254765], dtype=float32), array([ 0.02048296,  0.09543272,  0.04198248, ...,  0.39602429,
       -0.01850535,  0.12173899], dtype=float32), array([-0.06650621,  0.10761148, -0.41892618, ..., -0.35549027,
        0.02083908,  0.26113132], dtype=float32), array([ 0.17041151,  0.02062284, -0.07458932, ..., -0.05251388,
       -0.03330891,  0.

In [7]:
#SVM 


print(len(train_list))

clf = svm.SVC(kernel='rbf')
clf.fit(train_list, train_label_list)

train_errors=[]
train_errors.append(clf.score(train_list, train_label_list))
print(train_errors)




4800
[0.96729166666666666]


In [8]:
conceptLabelDict={}
errors=[]

def read_label(fname):
    with smart_open.smart_open(fname) as f:
        for i, line in enumerate(f):
            #get the id for each concept paragraph
            splitted = line.decode("iso-8859-1").split("\t")
            if len(splitted)==3:
                conceptLabelDict[splitted[1]] = splitted[2].replace("\r\n", "")
            else:
                errors.append(splitted)

label_file = "/home/hao/AnacondaProjects/MLOntology/ontClassLabels.txt"
read_label(label_file)

In [9]:
for i, (item, label) in enumerate(zip(test_list, test_label_list)):
    result = clf.predict([item])
    if result != label:
        print("index %d predicted label %s, but true label is %s" % (i, result, label))
        idpair = test_list_ids[i] 
        concept1 = conceptLabelDict[idpair[0]]
        concept2 = conceptLabelDict[idpair[1]]
        print("%s Concept Pairs: (%s --- %s)" % (idpair, concept1, concept2 ))

index 1 predicted label [0], but true label is 1
('94872003', '126835002') Concept Pairs: (neoplasm of uncertain behavior of ileum --- neoplasm of ileum)
index 30 predicted label [1], but true label is 0
('188729005', '426248008') Concept Pairs: (adult t-cell leukemia --- aleukemic lymphoid leukemia in remission)
index 48 predicted label [0], but true label is 1
('445104009', '396238001') Concept Pairs: (sum of proportion of positive staining neoplastic cells score and average intensity of staining score for hormone receptors using immunohistochemistry --- tumor measureable)
index 89 predicted label [0], but true label is 1
('276805005', '146801000119103') Concept Pairs: (leiomyoma of esophagus --- leiomyoma)
index 157 predicted label [1], but true label is 0
('262938004', '262902006') Concept Pairs: (secondary hemorrhage --- hematoma of urinary conduit)
index 166 predicted label [0], but true label is 1
('90678009', '41188003') Concept Pairs: (disorder of lip --- disorder of oral soft

In [10]:
# m3 = np.array((a,d))
# m3 = np.reshape(m3, 400, order='F')
print(len(test_list))
result = clf.predict(test_list)

print(result.size)


print(result[:29])
print(np.array(test_label_list[:29]))


                          

1200
1200
[1 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1]


In [11]:

err_ids=np.flatnonzero(result != test_label_list)

print(err_ids)
print(err_ids.size)

[   1   30   48   89  157  166  176  205  212  302  364  366  372  379  442
  505  525  588  597  646  681  689  721  764  801  831  833  835  850  851
  854  868  911  949 1013 1042 1073 1150 1192 1196]
40


In [12]:
from sklearn.metrics import accuracy_score, average_precision_score, f1_score, precision_score, recall_score, roc_auc_score
print(accuracy_score(result, test_label_list))
print(average_precision_score(result, test_label_list))

print(f1_score(result, test_label_list, average='macro') ) 

print(f1_score(result, test_label_list, average='micro')  )

print(f1_score(result, test_label_list, average='weighted') )

print(f1_score(result, test_label_list, average=None))

print(precision_score(result, test_label_list, average=None))
print(recall_score(result, test_label_list, average=None))

print(roc_auc_score(result, test_label_list, average=None))

0.966666666667
0.950172798216
0.966666574074
0.966666666667
0.96666675926
[ 0.96672213  0.96661102]
[ 0.96833333  0.965     ]
[ 0.96511628  0.96822742]
0.966671851909


In [ ]:
# train_label_list_m = np.eye(2)[train_label_list]
# test_label_list_m = np.eye(2)[test_label_list]
# print(test_label_list_m[10:100])

In [13]:
#CNN
import numpy as np
import math
from math import sqrt

import tensorflow as tf

'''
In the data, there are 2 classes and every class has 3000 samples and every sample has 512 features
The first 3000 samples are from class 0, second 3000 are from class 1
'''
# DATA_DIR = ''
CLASS_NUM = 2       #there are 2 classes
SPLIT_PERCENT = 0.8     #split the data into 80% for training and 20% for testing
FEATURE_NUM = 1024   
TRAIN_ITER = 1500    #the number of iterations for training
display_step = 100        #how many iterations to display the results


train_num = int(3000*SPLIT_PERCENT)     #the number of samples for training

train_feature = train_list      #training features (list of list)
train_y = train_label_list        #training lables    (list)
test_feature = test_list       #test features  (list of list)
test_y = test_label_list         #test labels    (list)


y_m = np.eye(2)[train_y]
test_y_m = np.eye(2)[test_y]

'''
y = wx+b        (vectors)
'''
#function to get variables 'w'
def weight_variable(shape, num):
    initial = tf.truncated_normal(shape, stddev=1/num)
    return tf.Variable(initial, name='weight')

#the bias 'b' in the equations
def bias_variable(shape, num):
    initial = tf.constant(0.0001, shape=shape)
    return tf.Variable(initial, name='bias')

#convolutional process
def conv1d(x, W):
    return tf.nn.conv1d(x, W, stride=1, padding='SAME')     #x: variable, w: weight, stride and padding (padding can be ignored currently) 

#pooling process
def max_pool_1x1(x, shape):
    x=tf.reshape(x,shape)       #it is transfered into four dimensions, but the other three are 1
    return tf.nn.max_pool(x, ksize=[1, 1, 4, 1],
                        strides=[1, 1, 2, 1], padding='SAME')

'''
The feature is 3 dimensional data.  [batch, length, channel] 
batch is usually ignored (for example there are 100 samples in a batch, so samples should not be modified mutually), length and channel are shown in the paper.
At first, the length is 512, and channel is 1.
Because our data are time series data, so length is enough, but for images, it may be [batch, length, width, channel]
'''
# the convolutional layer
def layer(features, f, input_n, channel, hidden_units, layer_index):
    """Construct a convolutional layer
    Args:
    features: Features placeholder, from the previous layer.
    f: the length
    input_n: Size of the features used in the convention.
    hidden_units: Size of the current hidden layer.
    layer_index: the index of layer
    Returns:
    hidden units: The unit output for the next layer.
    weights: the weights in the current hidden layer
    """
    # Hidden 1
    with tf.name_scope('hidden'+str(layer_index)) as scope:     # name scope may be ignored first
        with tf.name_scope("weight"):
            weights = weight_variable([input_n, channel, hidden_units], math.sqrt(f))

        with tf.name_scope("bias"):
            biases = bias_variable([hidden_units], math.sqrt(f))
    hidden = relu(conv1d(features, weights) + biases, 0.01)
    shape = [-1,1,f,hidden_units]
    h_pool1 = max_pool_1x1(hidden,shape)
    return h_pool1, weights

# fully connected layer, here the data are two dimension, [batch, length]
def densely_connect(features, input_n, hidden_units):
    """Construct a fully (densely) connected layer.
    Args:
    features: Features placeholder, from the previous layer.
    input_n: Size of units in the previous layer.
    hidden_units: Size of the current hidden layer.
    Returns:
    logits: The estimated output in last layer.
    weights: the weights in the hidden layer
    """
    with tf.name_scope('softmax_linear') as dense:
        with tf.name_scope("weight"):
            weights = weight_variable([input_n, hidden_units], math.sqrt(input_n))
        with tf.name_scope("bias"):
            biases = bias_variable([hidden_units], math.sqrt(input_n))
    logits = relu(tf.matmul(features, weights) + biases, 0.01)      # the matrix product operation
    return logits, weights

# dropout layer (it is not necessary)
# randomly set (1-keep_prob) percentage of units to be zero
def dropout(features, input_n, hidden_units, keep_prob):
    with tf.name_scope('dropout'):
        with tf.name_scope("weight"):
            weights = weight_variable([input_n, hidden_units], math.sqrt(input_n))
        with tf.name_scope("bias"):
            biases = bias_variable([hidden_units], math.sqrt(input_n))
    h_fc1_drop = tf.nn.dropout(features, keep_prob)
    drop_out = relu(tf.matmul(features, weights) + biases, 0.01)
    return drop_out

# calculate the loss in the neural network
def loss(logits, labels):
    """Calculates the loss from the logits and the labels.
    Args:
    logits: Logits tensor, float - [batch_size, NUM_CLASSES].
    labels: Labels tensor, int32 - [batch_size, NUM_CLASSES].
    Returns:
    loss: Loss tensor of type float.
    """
    with tf.name_scope("Loss"):
        labels = tf.to_int64(labels)
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
            labels=labels, logits=logits, name='xentropy')
    # tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))
    return tf.reduce_mean(cross_entropy, name='xentropy_mean')

def next_batch(data, label, num):
    """Generate the next batch randomly
    Args:
    data: training data.
    label: training label.
    num: the size in a batch
    Returns:
    next batch's training features and labels.
    """
    index = np.arange(len(data))
    np.random.shuffle(index)
#     train_feature = data[np.array(index)[0:num]]
#     train_label = label[np.array(index)[0:num]]
    train_feature_batch = [data[b] for b in index[0:num]]
    train_feature_batch = np.asarray(train_feature_batch)
    train_label_batch = [label[b] for b in index[0:num]]
    train_label_batch = np.asarray(train_label_batch)
    return train_feature_batch, train_label_batch

def relu(x, alpha=0., max_value=None):
    '''ReLU.
    alpha: slope of negative section.
    '''
    negative_part = tf.nn.relu(-x)
    x = tf.nn.relu(x)
    if max_value is not None:
        x = tf.clip_by_value(x, tf.cast(0., dtype=tf.float32),
                             tf.cast(max_value, dtype=tf.float32))
    x -= tf.constant(alpha, dtype=tf.float32) * negative_part
    return x

#define a session to run the model
sess = tf.InteractiveSession()

#place holders for training features and label
#None means the value is variable
x = tf.placeholder(tf.float32, shape=[None, FEATURE_NUM])
y_ = tf.placeholder(tf.float32, shape=[None, CLASS_NUM])

# decide whether it is training or testing, it is not used in our model, but it may be used
is_training = tf.placeholder(tf.bool)

#from [-1, 512, 1] -> [-1, 256, 32] -> [-1, 128, 64] -> [-1, 64, 64] -> [-1, 32, 64] -> [-1, 16, 64] -> [-1, 8, 64] -> [-1, 200]

#6 hidden layers
x_1 = tf.reshape(x, [-1,FEATURE_NUM,1])
h_pool0, w0 = layer(x_1, FEATURE_NUM, 15, 1, 32, 0)
h_pool0 = tf.reshape(h_pool0, [-1,512,32])
h_pool1, w1 = layer(h_pool0, 512, 10, 32, 64, 1)

h_pool1 = tf.reshape(h_pool1, [-1,256,64])
h_pool2, w2 = layer(h_pool1, 256, 10, 64, 64, 2)
h_pool2 = tf.reshape(h_pool2, [-1,128,64])
h_pool3, w3 = layer(h_pool2, 128, 10, 64, 64, 3)
h_pool3 = tf.reshape(h_pool3, [-1,64,64])
h_pool4, w4 = layer(h_pool3, 64, 5, 64, 64, 4)
h_pool4 = tf.reshape(h_pool4, [-1,32,64])
h_pool5, w5 = layer(h_pool4, 32, 5, 64, 64, 5)
h_pool5 = tf.reshape(h_pool5, [-1,16,64])
h_pool6, w6 = layer(h_pool5, 16, 5, 64, 64, 6)
h_pool6 = tf.reshape(h_pool6, [-1,8,64])

#densely connected: 200 units
h_pool_flat = tf.reshape(h_pool6, [-1, 8*64])
h_dc, w_d = densely_connect(h_pool_flat, 8*64, 200)

#dropout
keep_prob = tf.placeholder(tf.float32)
y_conv=dropout(h_dc, (int)(h_dc.get_shape()[1]), CLASS_NUM, keep_prob)


beta = 0.001
cross_entropy = loss(y_conv, y_)
loss = cross_entropy +beta*(tf.nn.l2_loss(w0) + tf.nn.l2_loss(w1)+tf.nn.l2_loss(w2)+tf.nn.l2_loss(w3)+tf.nn.l2_loss(w4)+tf.nn.l2_loss(w5)+tf.nn.l2_loss(w6)+tf.nn.l2_loss(w_d))  #L2 regularization
epsilon = 1e-5      # learning rate
train_step = tf.train.AdamOptimizer(epsilon).minimize(loss)     #optimization function, our goal is to minimize the loss

predict = tf.argmax(y_conv,1)   #the predicted class

# calculate the accuray, the corrected classified divided by the total size
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

#saver to save the training check point
# variables can be restored in a new model by 'saver.restore(sess, save_path)'
saver = tf.train.Saver()

sess.run(tf.global_variables_initializer())  #initialize the variables


for i in range(1,TRAIN_ITER):       #training iterations
    d, l = next_batch(train_feature, y_m, 100)      # get 100 samples in one batch
#     print("d size is %s, l size is %s "% (len(d), len(l)))
#     print("d size is %s, l size is %s "% (len(d[0]), len(l[0])))
    _, ls=sess.run([train_step,cross_entropy], feed_dict={x: d, y_: l, keep_prob: 1, is_training:True})     #run the train step (optimization function), the second one is just to show the loss in this iteration.   THE FEED dictionary is to feed the place holders which are needed in the optimization function.
    
    if i%display_step==0:
        print(_, i)
        acc = sess.run([accuracy], feed_dict={x: d, y_: l, keep_prob: 1, is_training:False})
        print("Train Loss:", ls, "Acc:", acc)

# sess.run  or tensor.eval are two ways
# get the accuracy in the testing data
print(accuracy.eval(session=sess, feed_dict={x:test_feature, y_:test_y_m, keep_prob: 1, is_training:False}))


# save the model results
save_path = saver.save(sess, "./model-noleaky.ckpt")
print("Model saved in file: %s" % save_path)

print("done")


None 100
Train Loss: 0.681625 Acc: [0.75]
None 200
Train Loss: 0.654703 Acc: [0.80000001]
None 300
Train Loss: 0.556615 Acc: [0.88999999]
None 400
Train Loss: 0.389736 Acc: [0.93000001]
None 500
Train Loss: 0.249503 Acc: [0.94999999]
None 600
Train Loss: 0.157865 Acc: [0.99000001]
None 700
Train Loss: 0.110269 Acc: [0.97000003]
None 800
Train Loss: 0.118554 Acc: [0.95999998]
None 900
Train Loss: 0.0839552 Acc: [0.99000001]
None 1000
Train Loss: 0.0927155 Acc: [0.95999998]
None 1100
Train Loss: 0.07362 Acc: [0.97000003]
None 1200
Train Loss: 0.0723841 Acc: [0.98000002]
None 1300
Train Loss: 0.0571115 Acc: [0.98000002]
None 1400
Train Loss: 0.023836 Acc: [1.0]
0.983333
Model saved in file: ./model-noleaky.ckpt
done


In [14]:

y_pred = sess.run(predict, feed_dict={x:test_feature, keep_prob:1, is_training:False})
print(y_pred[:20])
print(test_y[:20])


err_ids=np.flatnonzero(y_pred != test_y)

print(err_ids)
print(err_ids.size)
for err_id in err_ids:
    print("index %d predicted label %s, but true label is %s" % (err_id, y_pred[err_id], test_y[err_id]))
    idpair = test_list_ids[err_id] 
    concept1 = conceptLabelDict[idpair[0]]
    concept2 = conceptLabelDict[idpair[1]]
    print("%s Concept Pairs: (%s --- %s)" % (idpair, concept1, concept2 ))


[1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[  30  174  214  259  333  338  356  397  502  675  698  764  831  835  847
  852  865  962 1030 1081]
20
index 30 predicted label 1, but true label is 0
('188729005', '426248008') Concept Pairs: (adult t-cell leukemia --- aleukemic lymphoid leukemia in remission)
index 174 predicted label 1, but true label is 0
('264556007', '126966009') Concept Pairs: (occult neuropathy --- neoplasm of cranial nerve)
index 214 predicted label 1, but true label is 0
('126958000', '429033009') Concept Pairs: (neoplasm of cerebral ventricle --- malignant neoplasm of cerebrum)
index 259 predicted label 1, but true label is 0
('369514009', '369555007') Concept Pairs: (secondary malignant neoplasm of left fallopian tube --- malignant tumor involving right fallopian tube by separate metastasis from ovary)
index 333 predicted label 1, but true label is 0
('403869001', '403864006') Concept Pairs: (stucco ker

In [15]:
result = y_pred
test_label_list = test_y

from sklearn.metrics import accuracy_score, average_precision_score, f1_score, precision_score, recall_score, roc_auc_score
print(accuracy_score(result, test_label_list))
print(average_precision_score(result, test_label_list))

print(f1_score(result, test_label_list, average='macro') ) 

print(f1_score(result, test_label_list, average='micro')  )

print(f1_score(result, test_label_list, average='weighted') )

print(f1_score(result, test_label_list, average=None))

print(precision_score(result, test_label_list, average=None))
print(recall_score(result, test_label_list, average=None))

print(roc_auc_score(result, test_label_list, average=None))

0.983333333333
0.978870614035
0.98333259256
0.983333333333
0.983334074107
[ 0.98322148  0.98344371]
[ 0.97666667  0.99      ]
[ 0.98986486  0.97697368]
0.983419274538


In [ ]:

err_ids=np.flatnonzero(result != test_label_list)

print(err_ids)
print(err_ids.size)
for err_id in err_ids:
    idpair = test_list_ids[err_id] 
    print(idpair)
    concept1 = conceptLabelDict[idpair[0]]
    concept2 = conceptLabelDict[idpair[1]]
    print("(Concept 1 %s ---- Concept 2 %s)" % (concept1, concept2 ))


In [ ]:
plist = [[1,2,3], [3,4,5], [3,4,5], [5,6,7],[2,3,7]]

plist.extend([1]*4)
print(plist)

index = np.arange(5)

c = [plist[b] for b in index[:2]]
print(c)
c.extend([plist[b] for b in index[2:]])
print(c)

for i in range(3):      # i is the class index, for example, i==0 for class 0, i==1 for class 1 ...
    index = np.arange(30)     #generate numbers from 0 to 2999
    np.random.shuffle(index)        #shuffle the 3000 values
    index = [int(300*i+j) for j in index]
    print(index)
    print(np.array(index)[0:5])
    print(np.array(index)[5:])

In [ ]:
y=np.array([1,2,3])
y=np.append(y, [1]*4)
y= np.append(y, [0])
print(y)

In [ ]:

from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

In [ ]:
path = "D:/MLOntology/model1"

print(path)

model = gensim.models.Doc2Vec.load(path)
inferred_vector = model.infer_vector(['congenital', 'prolong', 'rupture', 'premature', 'membrane', 'lung'])
pprint(inferred_vector)
print(inferred_vector.size)
# pprint(model.docvecs.most_similar([inferred_vector], topn=20))


path = "D:/MLOntology/model0"

print(path)

model = gensim.models.Doc2Vec.load(path)
inferred_vector = model.infer_vector(['congenital', 'prolong', 'rupture', 'premature', 'membrane', 'lung'])
pprint(inferred_vector)
print(inferred_vector.size)


In [ ]:
model.docvecs['SENT_5690']

In [ ]:
from sklearn import svm
X = [[0, 0], [1, 1]]
y = [0, 1]
print(X)
print(y)
train_errors=[]
clf = svm.SVC()
clf.fit(X, y)  

train_errors.append(clf.score(X, y))
print(train_errors)
X_test=[[2,2]]
y_test = [1]
test_errors=[]
clf.predict(X_test)

test_errors.append(clf.score(X_test, y_test))
print(test_errors)

In [ ]:
from sklearn import svm

a= model.docvecs[0]
b= model.docvecs[1]
m1 = np.array((a, b))

# print(np.reshape(m1, 1024))
# print(np.reshape(m1, 400, order='F')) # two ways of reshape

c= model.docvecs[2]
d= model.docvecs[3]
print(c.shape)
print(d.shape)
print(c.shape[0]+d.shape[0])
m2 = np.array((c, d))

m1 = np.reshape(m1, 1024)
m2 = np.reshape(m2, 1024)
# m1 = np.reshape(m1, 1024, order='F')
# m2 = np.reshape(m2, 1024, order='F')

print(m1)

X = [m1, m2]
print(X)

XX = np.append(m1, m2)
print(XX)

y = [0, 1]
clf = svm.SVC(kernel='rbf')
clf.fit(X, y)

m3 = np.array((a,d))
m3 = np.reshape(m3, 1024, order='F')

result = clf.predict([m3])

print(result)

In [ ]:
import numpy as np

a=  np.array([1,2,3])
b = np.array([4,5,6])
m1 = np.array((a, b))
print(m1)

m2 = np.vstack((a, b)).T
print(m2)

In [ ]:
testDict={}
testDict[0] = ("a", "b")
print(testDict[0][1])

In [ ]:
a = np.array([1,1,0,1,0,1,0])
b = np.array([0,1,0,1,0,1,1])

np.flatnonzero(a!=b)